<a href="https://colab.research.google.com/github/AnnaSafaryan/Kursaches/blob/Evgrafova/%D0%9F%D1%80%D0%BE%D0%B1%D1%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install umap-learn

# Фразы

## Мало

In [2]:
phrases_short = [
    "from A to Z", "от А до Я",
    "from head to foot", "from head to feet", "с головы до ног",
    "on the off-chance", "на авось",
    "at a loss", "в растерянности",
    "on fire", "в ударе", "на высоте",
    "under lock and key", "за семью печатями",
    "out of the blue", "гром среди ясного неба",
    "in the flesh", "собственной персоной",
    "in the long run", "off and on"
]

In [3]:
# Полное совпадение по семантике и структуре
full_match_short = {
    "from A to Z": "от А до Я",
    "from head to foot": "с головы до ног",
    # "from head to feet": "с головы до ног",
    "on the off-chance": "на авось",
    "at a loss": "в растерянности"
}

# Частичное совпадение по семантике и структуре
partial_match_short = {
    "on fire": "в ударе",  # схлопывается
    "on fire": "на высоте",
    "under lock and key": "за семью печатями",
    "out of the blue": "гром среди ясного неба",
    "in the flesh": "собственной персоной"
}

# Фразеологизмы без аналогов в другом языке
no_match_short = {
    "in the long run": None,
    "off and on": None
}

## Много

In [4]:
import csv

In [5]:
prefix = '/content/drive/MyDrive/Colab Notebooks/Евграфова'

In [6]:
def parse_file(prefix, filename):
    full_match_list = []
    partial_match_list = []
    no_match_list = []

    with open(f"{prefix}/{filename}", mode="r", encoding="utf-8") as file:
        reader = csv.DictReader(file, delimiter="\t")
        for row in reader:
            if row['label'] == "+":
                full_match_list.append(row["phrase"])
            elif row['label'] == "-":
                partial_match_list.append(row["phrase"])
            else:
                no_match_list.append(row["phrase"])
    return full_match_list, partial_match_list, no_match_list

In [7]:
full_match_en, partial_match_en, no_match_en = parse_file(prefix, "en.tsv")
full_match_ru, partial_match_ru, no_match_ru = parse_file(prefix, "ru.tsv")

In [8]:
phrases_long = full_match_en + partial_match_en + no_match_en + full_match_ru + partial_match_ru + no_match_ru
full_match_long = dict(zip(full_match_en, full_match_ru))
partial_match_long = dict(zip(partial_match_en, partial_match_ru))
no_match_long = {k: None for k in partial_match_en + partial_match_ru}

# Импорты

In [9]:
from sentence_transformers import SentenceTransformer, util
from sklearn.cluster import KMeans
import numpy as np
import umap.umap_ as umap
from collections import defaultdict

# Эксперименты

## Косинусная близость

In [10]:
def encode(match_dict):
    phrases_1 = list(match_dict.keys())
    phrases_2 = list(match_dict.values())
    embeddings_1 = model.encode(phrases_1, normalize_embeddings=True)
    embeddings_2 = model.encode(phrases_2, normalize_embeddings=True)
    return phrases_1, phrases_2, embeddings_1, embeddings_2

In [11]:
def predict(phrases_1, phrases_2, embeddings_1, embeddings_2):
    ranks = []

    for i, phrase in enumerate(phrases_1):
        true_phrase = phrases_2[i]

        similarities = util.pytorch_cos_sim(embeddings_1[i], embeddings_2)
        best_match = phrases_2[similarities.argmax()]
        print(f"{phrase} = {true_phrase} → {best_match} (близость: {similarities.max():.2f})")
        # print(similarities, phrases_2)
        sorted_indices = similarities.argsort(descending=True).tolist()[0]
        # print(sorted_indices)
        ranks.append(sorted_indices.index(i) + 1)
        # print()
    print(sum(ranks) / len(ranks))  # идеально -- 1, чем больше ранг, тем хуже, т.к. правильный результат опускается ниже в рейтинге вариантов

In [12]:
def compare(match_dict):
    en_phrases, ru_phrases, en_embeddings, ru_embeddings = encode(match_dict)
    # print(en_phrases)
    # print(ru_phrases)
    # print()

    predict(en_phrases, ru_phrases, en_embeddings, ru_embeddings)
    print('\n__________________________________\n')
    predict(ru_phrases, en_phrases, ru_embeddings, en_embeddings)

In [13]:
model = SentenceTransformer("sentence-transformers/LaBSE")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


###  Мало

In [14]:
compare(full_match_short)

from A to Z = от А до Я → от А до Я (близость: 0.91)
from head to foot = с головы до ног → с головы до ног (близость: 0.95)
on the off-chance = на авось → на авось (близость: 0.68)
at a loss = в растерянности → в растерянности (близость: 0.77)
1.0

__________________________________

от А до Я = from A to Z → from A to Z (близость: 0.91)
с головы до ног = from head to foot → from head to foot (близость: 0.95)
на авось = on the off-chance → on the off-chance (близость: 0.68)
в растерянности = at a loss → at a loss (близость: 0.77)
1.0


In [15]:
compare(partial_match_short)

on fire = на высоте → на высоте (близость: 0.62)
under lock and key = за семью печатями → за семью печатями (близость: 0.46)
out of the blue = гром среди ясного неба → на высоте (близость: 0.51)
in the flesh = собственной персоной → на высоте (близость: 0.52)
1.75

__________________________________

на высоте = on fire → on fire (близость: 0.62)
за семью печатями = under lock and key → out of the blue (близость: 0.47)
гром среди ясного неба = out of the blue → out of the blue (близость: 0.46)
собственной персоной = in the flesh → in the flesh (близость: 0.47)
1.25


###  Много

In [16]:
compare(full_match_long)

the alpha and omega = альфа и омега → альфа и омега (близость: 0.93)
from A to Z = от А до Я → от А до Я (близость: 0.91)
from head to feet = с головы до ног → с головы до ног (близость: 0.95)
on the off-chance = на авось → на авось (близость: 0.68)
at a loss = в растерянности → в растерянности (близость: 0.77)
as the call, so the echo = как аукнется, так и откликнется → как аукнется, так и откликнется (близость: 0.67)
on the run = на бегу → на бегу (близость: 0.94)
to infinity = до бесконечности → до бесконечности (близость: 0.94)
with flair = с блеском → с блеском (близость: 0.76)
godforsaken = богом забытый → богом забытый (близость: 0.65)
more or less = более или менее → более или менее (близость: 0.97)
stardom sickness = звёздная болезнь → звёздная болезнь (близость: 0.90)
powder keg = пороховая бочка → пороховая бочка (близость: 0.58)
1.0

__________________________________

альфа и омега = the alpha and omega → the alpha and omega (близость: 0.93)
от А до Я = from A to Z → from 

In [17]:
compare(partial_match_long)

on fire = в ударе → в ажуре (близость: 0.65)
under lock and key = за семью печатями → за семью печатями (близость: 0.46)
out of the blue = гром среди ясного неба → в ажуре (близость: 0.57)
in the flesh = собственной персоной → в ажуре (близость: 0.64)
bark the wrong tree = не по адресу → не по адресу (близость: 0.41)
tiptop = в ажуре → в ударе (близость: 0.52)
highlight key points = расставить акценты → расставить акценты (близость: 0.69)
dead weight = тяжёлая артиллерия → до гроба (близость: 0.48)
better a small fish than an empty dish = на безрыбье и рак рыба → на безрыбье и рак рыба (близость: 0.49)
warm and fuzzy = белый и пушистый → белый и пушистый (близость: 0.70)
for a song = за бесценок → за бесценок (близость: 0.53)
fool around = играть в бирюльки → в ажуре (близость: 0.54)
god knows = видит бог → видит бог (близость: 0.80)
till the grave = до гроба → до гроба (близость: 0.94)
1.7857142857142858

__________________________________

в ударе = on fire → on fire (близость: 0.65)

## Кластеризация

In [18]:
model = SentenceTransformer("sentence-transformers/LaBSE")

In [19]:
# Кластеризация (возьмём 3 кластера, как в классификации)
def clustering(embeddings):
    num_clusters = 3
    clustering_model = KMeans(n_clusters=num_clusters, random_state=42)
    clustering_model.fit(embeddings)
    cluster_labels = clustering_model.labels_
    return cluster_labels

In [20]:
def list_clusters(cluster_labels, phrases):
    cluster_dict = defaultdict(list)
    for phrase, label in zip(phrases, cluster_labels):
        cluster_dict[label].append(phrase)
    return cluster_dict

In [21]:
def predict(phrases):
    embeddings = model.encode(phrases)
    embeddings = umap.UMAP(n_components=2, random_state=42).fit_transform(embeddings)
    cluster_labels = clustering(embeddings)
    return cluster_labels

### Мало

In [22]:
%%time
cluster_labels = predict(phrases_short)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


CPU times: user 17.5 s, sys: 1.61 s, total: 19.1 s
Wall time: 28.7 s


In [23]:
list_clusters(cluster_labels, phrases_short)

defaultdict(list,
            {1: ['from A to Z',
              'от А до Я',
              'from head to foot',
              'from head to feet',
              'с головы до ног',
              'under lock and key',
              'off and on'],
             0: ['on the off-chance',
              'at a loss',
              'в растерянности',
              'в ударе',
              'за семью печатями',
              'out of the blue',
              'гром среди ясного неба'],
             2: ['на авось',
              'on fire',
              'на высоте',
              'in the flesh',
              'собственной персоной',
              'in the long run']})

### Много

In [24]:
%%time
cluster_labels = predict(phrases_long)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


CPU times: user 3.76 s, sys: 25.4 ms, total: 3.79 s
Wall time: 4.17 s


In [25]:
list_clusters(cluster_labels, phrases_long)

defaultdict(list,
            {2: ['the alpha and omega',
              'as the call, so the echo',
              'godforsaken',
              'more or less',
              'better a small fish than an empty dish',
              'warm and fuzzy',
              'god knows',
              'альфа и омега',
              'как аукнется, так и откликнется',
              'богом забытый',
              'более или менее',
              'не по адресу',
              'на безрыбье и рак рыба',
              'белый и пушистый',
              'видит бог',
              'как в аптеке',
              'не ахти сколько',
              'не ахти какой',
              'как баран на новые ворота',
              'ни бе ни ме ни кукареку',
              'пришла беда – отворяй ворота',
              'белены объелся',
              'бережёного бог бережёт',
              'ни бельмеса',
              'жёлтый билет',
              'бог знает как',
              'на бога надейся, а сам не плошай'],
             1